In [ ]:
######################################################
# Juypter
######################################################
%load_ext autoreload
%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))

######################################################
# Versions
######################################################
import sys
print("Python: {0}".format(sys.version))
import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

# MusicBrainz

In [ ]:
%load_ext autoreload
%autoreload
from dbArtistsMusicBrainz import dbArtistsMusicBrainz
from dbArtistsParse import dbArtistsPrimary, dbArtistsCredit, dbArtistsExtra
from dbArtistsParse import dbArtistsAssertExtra

In [ ]:
for modVal in range(100):
    dbAP = dbArtistsPrimary(dbArtistsMusicBrainz())
    dbAP.parse(modVal, previousDays=1, force=False)

for modVal in range(100):
    dbAP = dbArtistsExtra(dbArtistsMusicBrainz())
    dbAP.parse(modVal)
    
for modVal in range(100):
    dbAP = dbArtistsPrimary(dbArtistsMusicBrainz())
    dbAP.createArtistMetadata(modVal)
    dbAP.createAlbumMetadata(modVal)

In [ ]:
%load_ext autoreload
%autoreload
from dbArtistsDiscogs import dbArtistsDiscogs
from dbArtistsParse import dbArtistsPrimary, dbArtistsCredit, dbArtistsExtra
from dbArtistsParse import dbArtistsAssertExtra, dbArtistsUnofficial, dbArtistsAssertUnofficial

In [ ]:
for modVal in range(100):
    dbAP = dbArtistsPrimary(dbArtistsDiscogs())
    dbAP.parse(modVal, previousDays=3, force=False)
    
for modVal in range(100):
    break
    dbAP = dbArtistsExtra(dbArtistsDiscogs())
    dbAP.parse(modVal, previousDays=10, force=True)
    
for modVal in range(100):
    break
    dbAP = dbArtistsUnofficial(dbArtistsDiscogs())
    dbAP.parse(modVal, previousDays=10, force=True)
    
for modVal in range(100):
    dbAP = dbArtistsPrimary(dbArtistsDiscogs())
    dbAP.createArtistMetadata(modVal)
    dbAP.createAlbumMetadata(modVal)

In [ ]:
%load_ext autoreload
%autoreload
from dbArtistsDiscogs import dbArtistsDiscogs
from dbArtistsParse import dbArtistsPrimary, dbArtistsCredit, dbArtistsExtra
from dbArtistsParse import dbArtistsAssertExtra, dbArtistsUnofficial, dbArtistsAssertUnofficial

In [ ]:
for modVal in range(95,100):
    dbAP = dbArtistsPrimary(dbArtistsDiscogs())
    dbAP.createArtistMetadata(modVal)
    dbAP.createAlbumMetadata(modVal)

# Get Cross Links

In [ ]:
from dbArtistsMusicBrainz import dbArtistsMusicBrainz

In [ ]:
dbAP = dbArtistsMusicBrainz()
disc = dbAP.disc
db = disc.getArtistNameToIDData()

In [ ]:
extData = {}

In [ ]:
for modVal in range(100):
    print(modVal,end="\t")
    dbdata = disc.getArtistsDBModValData(modVal)
    for artistID, artistData in dbdata.items():
        artistName = artistData.artist.name        
        ext = artistData.profile.external
        extData[(artistID,artistName)] = ext
    print(len(extData))

In [ ]:
from dbArtistsAllMusic import dbArtistsAllMusic
from dbArtistsDiscogs import dbArtistsDiscogs
from dbArtistsLastFM import dbArtistsLastFM
from dbArtistsGenius import dbArtistsGenius

In [ ]:
from collections import Counter
cnt = Counter()

extDBRawLinkValues = {}# {"musicbrainz": {}}


for key,data in extData.items():
    for db,url in data.items():
        if db == "all-relationships": # MusicBrainz's data
            continue
        if url.startswith("http") is False: # If it's somehow otherwise internal
            continue
        if db == "home-favicon": # The artist homepage
            continue
        #print("\t",db,'\t',url)
        dbname = db.split('-')[0] ## MusicBrainz adds some extra stuff
        if dbname in ["facebook", "twitter", "instagram", "imdb", "myspace"]: # Social sites
            continue
        if dbname in ["viaf"]: # Virtual Internet Authority File (something about different spellings, maybe?)
            continue
        if dbname in ["dnb", 'loc', 'worldcat', 'bnfcatalogue']: # German National Library, Library of Congress, World Catalog, French National Library
            continue
        if dbname in ["songkick"]: # Information about upcoming live shows
            continue
        if dbname in ["decoda"]: # Messy website
            continue
        if dbname in ["secondhandsongs"]: # Website about song covers
            continue
        if dbname in ["setlistfm"]: # Website with live show set lists
            continue

        cnt[dbname] += 1
        if dbname == "viaf":
            print(db,url)
            break

        if extDBRawLinkValues.get(key) is None:
            extDBRawLinkValues[key] = {}
        extDBRawLinkValues[key][dbname] = url

In [ ]:
%autoreload
dutils = {}
from dbUtils import utilsAllMusic, utilsDeezer, utilsDiscogs, utilsGenius, utilsLastFM, utilsMusicBrainz, utilsRateYourMusic
dutils["allmusic"] = utilsAllMusic()
dutils["deezer"] = utilsDeezer()
dutils["discogs"] = utilsDiscogs()
dutils["genius"] = utilsGenius()
dutils["lastfm"] = utilsLastFM()
dutils["musicbrainz"] = utilsMusicBrainz()
dutils["rateyourmusic"] = utilsRateYourMusic()

In [ ]:
rymToGet = {}
dbLinks = {"musicbrainz": {}}
dbIDToURLLookup = {}
from collections import Counter
dbCnt = Counter()
for i,(key,value) in enumerate(extDBRawLinkValues.items()):
    mbID = key[0]
    #print(key[0],key[1])
    for db,url in value.items():
        
        if db == "rateyourmusic":
            rymToGet[url] = key
            continue
            
        
        if dutils.get(db) is not None:
            artistID = dutils[db].getArtistID(url, debug=False)
            if artistID is not None:
                if dbIDToURLLookup.get(db) is None:
                    dbIDToURLLookup[db] = {}
                dbIDToURLLookup[db][artistID] = url
                
                if dbLinks["musicbrainz"].get(mbID) is None:
                    dbLinks["musicbrainz"][mbID] = {}
                dbLinks["musicbrainz"][mbID][db] = artistID
                
                if dbLinks.get(db) is None:
                    dbLinks[db] = {}
                if dbLinks[db].get(artistID) is None:
                    dbLinks[db][artistID] = [{"musicbrainz": mbID}]
                else:
                    #print("Multiple links to {0}-{1} from MB-{2} ({3})".format(db,artistID,mbID,key[1]))
                    dbLinks[db][artistID].append({"musicbrainz": mbID})
        else:
            dbCnt[db] += 1
            if db == "setlistfm":
                print(url)
                1/0

## Spotify Test

In [ ]:
url="https://open.spotify.com/artist/0IecGJbdBeYSOVtSPRehh5"
savename="spotify.test.p"
from webUtils import downloadURL
data, code = downloadURL(url)
#help(getWebData)

## Metal-Archives Test

In [ ]:
url="https://www.metal-archives.com/bands/The_Bleeding/3540370833"
savename="metalarchives.test.p"
from webUtils import downloadURL
data, code = downloadURL(url)
#help(getWebData)

In [ ]:
from ioUtils import saveFile
saveFile(idata=dbLinks, ifile="dbLinks.p")
saveFile(idata=dbIDToURLLookup, ifile="dbIDToURLLookup.p")
saveFile(idata=rymToGet, ifile="dbRYMToGet.p")

## Not Used DBs

In [ ]:
dbCnt.most_common()

## Get missing URLs from DBs

In [ ]:
from ioUtils import getFile
dbLinks = getFile("dbLinks.p")
dbIDToURLLookup = getFile("dbIDToURLLookup.p")
rymToGet = getFile("dbRYMToGet.p")

In [ ]:
from dbArtistsDeezer import dbArtistsDeezer
dbDeez = dbArtistsDeezer()
deezRef = dbDeez.disc.getMasterSlimArtistDiscogsDB()

In [ ]:
deezToGet = Series(list(dbLinks['deezer'].keys()))
IDsToGet  = deezToGet[~deezToGet.isin(deezRef.index)]

In [ ]:
from time import sleep
from fsUtils import isFile
from random import random
n = len(IDsToGet)
for i,artistID in enumerate(IDsToGet):
    savename = dbDeez.dutils.getArtistSavename(artistID)
    url      = dbDeez.getArtistURL(artistID)
    if not isFile(savename):
        print("{0: >3}/{1: <5}\t{2}\t{3}".format(i,n,savename,url))
        try:
            dbDeez.dutils.downloadArtistURL(url=url, savename=savename, force=True)
        except:
            pass
        sleep(2+random())

In [ ]:
deezRef

In [ ]:
dbDeez.getArtistURL('4190')

## Get MusicBrainz Extrapages

In [ ]:
%load_ext autoreload
%autoreload
from dbArtistsMusicBrainz import dbArtistsMusicBrainz
from dbArtistsParse import dbArtistsPrimary, dbArtistsCredit, dbArtistsExtra
from dbArtistsParse import dbArtistsAssertExtra, dbArtistsUnofficial, dbArtistsAssertUnofficial
dbMB = dbArtistsMusicBrainz()
for modVal in range(0):
    #if True:
    #try:
        #lfm.assertDBModValExtraData(modVal, minPages=10, maxPages=20, test=False)
        dbMB.assertDBModValExtraData(modVal, minPages=2, maxPages=30000000, test=False, clean=True)
    #except:
    #    pass

## Get RateYourMusic's That Aren't Known

In [ ]:
from dbArtistsRateYourMusic import dbArtistsRateYourMusic
from dbArtistsMusicBrainz import dbArtistsMusicBrainz
from pandas import DataFrame, Series

dbA = dbArtistsRateYourMusic()
dbMB = dbArtistsMusicBrainz()

In [ ]:
mdbAlbs = dbMB.disc.getMasterSlimArtistAlbumsDiscogsDB()

def getNumAlbums(x):
    numAlbums = sum([len(y) for y in x.values()])
    return numAlbums

mbNumAlbums = DataFrame(mdbAlbs["Albums"].apply(getNumAlbums))
mbNumAlbums.columns = ["MBNumAlbums"]
print("Size = {0}".format(mbNumAlbums.shape))
mbNumAlbums.head()

In [ ]:
mbNumAlbums = mbNumAlbums.reset_index(drop=False)
mbNumAlbums.columns = ["MBID", "MBNumAlbums"]
mbNumAlbums.head()

In [ ]:
rymAlbs = dbA.disc.getMasterSlimArtistAlbumsDiscogsDB()

In [ ]:
rymNumAlbums = DataFrame(rymAlbs["Albums"].apply(getNumAlbums))
rymNumAlbums.columns = ["RYMNumAlbums"]
print("Size = {0}".format(rymNumAlbums.shape))
rymNumAlbums.head()

In [ ]:

rymRef = dbA.disc.getMasterSlimArtistDiscogsDB()
rymRef = rymRef.join(rymNumAlbums)
rymRef["Ref"] = rymRef["Ref"].apply(lambda x: x.replace("-", "_"))
rymRef["Ref"] = rymRef["Ref"].apply(lambda x: x.replace("http:", "https:"))

In [ ]:
rymRefLookup = DataFrame(Series({row["Ref"]: {"RYMArtistID": idx, "RYMNumAlbums": row["RYMNumAlbums"]} for idx,row in rymRef.iterrows()}))
rymRefDF = rymRefLookup[0].apply(Series)
rymRefDF.index.name = "RYMURL"
print("Size = {0}".format(rymRefDF.shape))
rymRefDF.head()

In [ ]:
rymRef[rymRef['CleanDiscArtist'].str.contains("Johann Sebastian Bach")]

In [ ]:
rymRefDF[rymRefDF.index == "https://rateyourmusic.com/artist/johann_sebastian_bach"]

In [ ]:
rymToGetDF = DataFrame(Series(rymToGet))
rymToGetDF.columns   = ["MusicBrainzKey"]
rymToGetDF = rymToGetDF["MusicBrainzKey"].apply(Series)
rymToGetDF.columns   = ["MBID", "MBArtistName"]
rymToGetDF.reset_index(inplace=True, drop=False)
rymToGetDF.columns   = ["RYMURL", "MBID", "MBArtistName"]
rymToGetDF["RYMURL"] = rymToGetDF["RYMURL"].apply(lambda x: x.replace("-", "_"))
rymToGetDF["RYMURL"] = rymToGetDF["RYMURL"].apply(lambda x: x.replace("http:", "https:"))
rymToGetDF.head()

In [ ]:
rymToGetDFWithAlbums = rymToGetDF.merge(mbNumAlbums, on='MBID')
rymToGetDFWithAlbums.head()

In [ ]:
rymToGetDFWithAlbums.index = rymToGetDFWithAlbums["RYMURL"]
rymToGetDFWithAlbums.drop(['RYMURL'], inplace=True, axis=1)
rymToGetDFWithAlbums.head()

In [ ]:
resultDF = rymToGetDFWithAlbums.join(rymRefDF)
resultDF.head()

#### Results That I Have

In [ ]:
resultDF[~resultDF["RYMArtistID"].isna()].sort_values(by='MBNumAlbums', ascending=False)

#### Results That I Don't Have

In [ ]:
unknownDF = resultDF[resultDF["RYMArtistID"].isna()].sort_values(by='MBNumAlbums', ascending=False)

unknownRYMURLs = dict(zip(unknownDF.index, unknownDF['MBNumAlbums']))
unknownDF.head()

In [ ]:
hardcutoff = 60
softcutoff = 10
for link,n in unknownRYMURLs.items():
    if n >= hardcutoff:
        continue
        
    if link.find("%") == -1:        
        ## No Non-ascii
        if n >= 41:
            continue
        if n < 40:
            break
        print(n,'\t',link)
    else:
        ## Contains Non-ascii
        if n >= softcutoff:
            continue
        print(n,'\t',link)


In [ ]:
from artistLastFM import artistLastFM
afm = artistLastFM()
retval = afm.getData("/Volumes/Piggy/Discog/artists-lastfm/1/42055961701.p")
retval.show()

In [ ]:
#db["Dave Matthews Band"]

In [ ]:
ifile = "/Volumes/Piggy/Discog/artists-musicbrainz/17/162010259733035707919464750326182263717.p"

In [ ]:
retval = dbAP.artist.getData(ifile)
retval.profile.external

In [ ]:
from webUtils import getHTML
bsdata = getHTML(ifile)

In [ ]:
bsdata